# Tiziano Borrelli
# Coursera Applied Data Science Capstone 

## Week 3 - Exercise 1

### Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import requests
from bs4 import BeautifulSoup
import os

print('Libraries successfully imported.')

Libraries successfully imported.


### Load Data and Create dataframe 

In [4]:
#Import Data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,features="lxml")
table=soup.find('table')
rows=table.findAll('tr')
data={}
index=0
for r in rows:
    row_data=r.findAll('td')
    if row_data:
        data.update({
            index:[
                #Postal Code
                row_data[0].text.replace("\n",""), 
                #Borough
                row_data[1].text.replace("\n",""),
                #Neighbourhood
                row_data[2].text.replace("\n","")
            ]
        })
        index+=1

### Exercize 1 > The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
#Data Frame
columns= ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame.from_dict(data=data, orient='index', columns=columns)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


###  Exercize 2 > Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df=df[df['Borough']!='Not assigned']
df.head(15)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


### Exercize 4 > If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(15)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


### Exercize 3 > More than one neighborhood can exist in one postal code area.

In [8]:
result = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df_new=result.reset_index()
df_new.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


###  Exercize 5 > Create new dataframe and shape

In [9]:
df_new.shape

(103, 3)

## Week 3 - Exercise 2

### Download and Explore Dataset

In [62]:
!"C:\Program Files (x86)\GnuWin32\bin\wget" -q -O toronto_data.json https://geo.nyu.edu/download/file/nyu-2451-36098-geojson.json --no-check-certificate
print('Data downloaded!')
with open('toronto_data.json') as json_data:
    toronto_data = json.load(json_data)

#Data 
#toronto_data
neighborhoods_data = toronto_data['features']
#print(neighborhoods_data)
neighborhoods_data[0]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
# fill the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['f_code'] 
    neighborhood_name = data['properties']['nam']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head() 

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc


Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,AL015,UNK,47.461189,-80.282936
1,AL015,UNK,47.304363,-80.303383
2,AL015,UNK,47.164413,-80.065041
3,AL015,UNK,47.080826,-80.093231
4,AL015,UNK,47.012661,-80.114243


#### Use geopy library to get the latitude and longitude values of Toronto

In [63]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [64]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Define Foursquare Credentials and Version

In [51]:
CLIENT_ID = '4HHHFRUHK13TP1XW1YBT4A4A2TNYZKW0WD41CUZAX5CEUNDS' # your Foursquare ID
CLIENT_SECRET = 'XA1IYGJKDA0ZOY3NUW441UFPAEZCGY2L1JMUKHK0IIXWD41Y' # your Foursquare Secret
VERSION = '20200405' # Foursquare API version

#### Now, let's get the top 100 venues that are in Tornto within a radius of 500 meters.

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Indigo,Bookstore,43.653515,-79.380696


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Get all neighborhoods in Toronto

In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
WEYMONT
UNK#ELECTRIC FACILITY
DESSANE
VANDRY
FERGUSON
DUPLESSIS
RAPIDE-BLANC(GARE)
CRESSMAN
FITZPATRICK
DOHENY
SUMMIT
CLUB-GREGOIRE
UNK
LAC-AUX-PERLES
SANFORD
STADACONA
CLUB-IROQUOIS
LAC-MALOUIN
UNK
CLUB-JACQUES-CARTIER
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
BURWASH STATION
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK#ELECTRIC FACILITY
UNK#ELECTRIC FACILITY
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
MARELAN
LALANDE
CARILLON-STATION
VANKLEEK HILL STATION
ALFRED STATION
COMO-STATION
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
UNK
FLESHERTON STATION
PINKERTON STATION
MCWILLIAMS STATION
UNK
UNK
UNK#RAILWAY STATION
UNK#RAILWAY STATION
MILLBRIDGE STATION
MARMONA STATION
BLAIRTON STATION
UNK
CONSECON STATION
ROBLINDALE
FINDLEY
UNK
FORFAR STATION
UNK#ELECTRIC FACILITY
GANANOQUE JUNCTION
UNK
UNK
BUCKLEY CAMP
STATE FORESTRY CAMP
STATE RA

In [81]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AERIAL TRAMWAY,4,4,4,4,4,4
BASE STATION,4,4,4,4,4,4
BEACON,1,1,1,1,1,1
CAMP,2,2,2,2,2,2
COMO-STATION,2,2,2,2,2,2
DRIVE-IN THEATER,2,2,2,2,2,2
GANANOQUE JUNCTION,1,1,1,1,1,1
HOSPITAL,4,4,4,4,4,4
JOSEPH SMITH MONUMENT,1,1,1,1,1,1


In [83]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


## Exercise 3 - Cluster Neighborhoods

In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cable Car,Café,Campground,Candy Store,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Coffee Shop,College Academic Building,College Cafeteria,College Theater,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Duty-free Shop,Electronics Store,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Lawyer,Library,Lingerie Store,Liquor Store,Marijuana Dispensary,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Museum,New American Restaurant,Nightlife Spot,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stables,State / Provincial Park,Steakhouse,Student Center,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toll Plaza,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Weight Loss Center,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,UNK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,UNK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [90]:
print('old size {}'.format(toronto_onehot.shape))
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
print('new size {}'.format(toronto_grouped.shape))

old size (869, 231)
new size (27, 231)


### Print each neighborhood along with the top 5 most common venue

In [92]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AERIAL TRAMWAY----
                   venue  freq
0                    Pub  0.25
1                 Resort  0.25
2                    Gym  0.25
3               Mountain  0.25
4  Portuguese Restaurant  0.00


----BASE STATION----
                          venue  freq
0                    Restaurant  0.25
1                      Platform  0.25
2                         Trail  0.25
3  Theme Park Ride / Attraction  0.25
4             Accessories Store  0.00


----BEACON----
               venue  freq
0              Trail   1.0
1  Accessories Store   0.0
2          Pet Store   0.0
3           Mountain   0.0
4      Movie Theater   0.0


----CAMP----
               venue  freq
0         Campground   1.0
1  Accessories Store   0.0
2           Pharmacy   0.0
3      Movie Theater   0.0
4             Museum   0.0


----COMO-STATION----
                        venue  freq
0               Boat or Ferry   0.5
1  Construction & Landscaping   0.5
2                   Pet Store   0.0
3                

### Store into a pandas dataframe

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AERIAL TRAMWAY,Pub,Mountain,Resort,Gym,Dive Bar,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop
1,BASE STATION,Trail,Platform,Theme Park Ride / Attraction,Restaurant,Yoga Studio,Electronics Store,Doctor's Office,Dog Run,Donut Shop,Duty-free Shop
2,BEACON,Trail,Yoga Studio,Dive Bar,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop
3,CAMP,Campground,Yoga Studio,Doctor's Office,Food,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Factory
4,COMO-STATION,Boat or Ferry,Construction & Landscaping,Dog Run,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### Cluster Neighborhoods

In [128]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(100) # check the last columns!

df = toronto_merged.copy()
df['Cluster Labels'] = '1'
df.head()



,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AL015,UNK,47.461189,-80.282936,1,Clothing Store,Farm,Lake,Café,Hotel,Trail,American Restaurant,Baseball Field,Pizza Place,Construction & Landscaping
1,AL015,UNK,47.304363,-80.303383,1,Clothing Store,Farm,Lake,Café,Hotel,Trail,American Restaurant,Baseball Field,Pizza Place,Construction & Landscaping
2,AL015,UNK,47.164413,-80.065041,1,Clothing Store,Farm,Lake,Café,Hotel,Trail,American Restaurant,Baseball Field,Pizza Place,Construction & Landscaping
3,AL015,UNK,47.080826,-80.093231,1,Clothing Store,Farm,Lake,Café,Hotel,Trail,American Restaurant,Baseball Field,Pizza Place,Construction & Landscaping
4,AL015,UNK,47.012661,-80.114243,1,Clothing Store,Farm,Lake,Café,Hotel,Trail,American Restaurant,Baseball Field,Pizza Place,Construction & Landscaping


In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters